In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from PIL import Image
import os
import csv
from skimage import io
work_dir = "/kaggle/input/birds-22wi/birds"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
class TestImageDataSet(Dataset):

    def __init__(self, label_csv_file, root_dir, fileNames, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.true_label_csv = label_csv_file
        self.true_label_map = {}
        self.fileNameList = fileNames
        self.root_dir = root_dir
        self.transform = transform
#         self.parseLabelsToDict()
        
    def parseLabelsToDict(self):
        with open(self.true_label_csv, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                path = row["path"]
                classes = row["class"]
                self.true_label_map[path] = classes
            csvfile.close()
            
    def __len__(self):
        return len(self.fileNameList)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.fileNameList[idx])
        image = Image.open(img_name).convert('RGB')
        
        
        item = [image, self.fileNameList[idx]]
        if self.transform:
            item[0] = self.transform(item[0])
        return item[0], item[1]
    
class TrainImageDataSet(Dataset):

    def __init__(self, label_csv_file, root_dir, fileNames, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.true_label_csv = label_csv_file
        self.true_label_map = {}
        self.fileNameList = fileNames
        self.root_dir = root_dir
        self.transform = transform
        self.parseLabelsToDict()
        
    def parseLabelsToDict(self):
        with open(self.true_label_csv, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                path = row["path"]
                classes = row["class"]
                self.true_label_map[path] = int(classes)
            csvfile.close()
            
    def __len__(self):
        return len(self.fileNameList)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = os.path.join(self.root_dir,
                                self.fileNameList[idx])
        image = Image.open(img_name).convert('RGB')
        
        imageName = self.fileNameList[idx].split("/")[-1]
        item = [image, self.true_label_map[imageName]]
        if self.transform:
            item[0] = self.transform(item[0])
        return item[0], torch.tensor(item[1])

In [ ]:
def get_birds_data():
    # Data augmentation transformations. Not for Testing!
    transform_train = transforms.Compose([
        transforms.Resize(256), # Takes images smaller than 64 and enlarges them
        transforms.RandomCrop(256, padding=4, padding_mode='edge'), # Take 64x64 crops from 72x72 padded images
        transforms.RandomHorizontalFlip(),    # 50% of time flip image along y-axis
        transforms.ToTensor(),
    ])

    transform_test = transforms.Compose([
        transforms.Resize([256,256]),
        transforms.ToTensor(),
    ])

    trainFolderNames = os.listdir("/kaggle/input/birds-22wi/birds/train")
    trainFileNames = []
    for name in trainFolderNames:
        imageNames = (os.listdir("/kaggle/input/birds-22wi/birds/train/" + name))
        trainFileNames += ["/kaggle/input/birds-22wi/birds/train/" + name +"/"+ imageName for imageName in imageNames]
    trainset = TrainImageDataSet(label_csv_file="/kaggle/input/birds-22wi/birds/labels.csv", 
                               root_dir='/kaggle/input/birds-22wi/birds/train', fileNames=trainFileNames, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

    testFileNames = os.listdir("/kaggle/input/birds-22wi/birds/test/0")
    testset = TestImageDataSet(label_csv_file="/kaggle/input/birds-22wi/birds/labels.csv", 
                               root_dir='/kaggle/input/birds-22wi/birds/test/0', fileNames=testFileNames, transform=transform_test)
#     testset = torchvision.datasets.ImageFolder(root='/kaggle/input/birds-22wi/birds/test/', transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2, pin_memory=False)
    return {'train': trainloader, 'test': testloader}

data = get_birds_data()

In [ ]:
dataiter = iter(data['train'])
images, labels = dataiter.next()
images = images[:8]
print(len(images))
print(images.size())

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print("Labels:" + ' '.join('%9s' % labels[j] for j in range(8)))

flat = torch.flatten(images, 1)
print(images.size())
print(flat.size())

In [ ]:
#WITHOUT BIAS
class ConvBNNet(nn.Module):
    def __init__(self):
        super(ConvBNNet, self).__init__() # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 64, 3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(64)

        self.conv4 = nn.Conv2d(64, 128, 3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(128)

        self.conv5 = nn.Conv2d(128, 256, 3, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(256)
        
        self.conv6 = nn.Conv2d(256, 512, 3, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(512)
        
        self.conv7 = nn.Conv2d(512, 1024, 3, padding=1, bias=False)
        self.bn7 = nn.BatchNorm2d(1024)

        self.fc1 = nn.Linear(1024, 555)

    def forward(self, x):
        # Input 256x256x3

        x = F.max_pool2d(F.relu(self.bn1(self.conv1(x))), kernel_size=2, stride=2) # 128x128x16
        x = F.max_pool2d(F.relu(self.bn2(self.conv2(x))), kernel_size=2, stride=2) # 64x64x32
        x = F.max_pool2d(F.relu(self.bn3(self.conv3(x))), kernel_size=2, stride=2) # 32x32x64
        x = F.max_pool2d(F.relu(self.bn4(self.conv4(x))), kernel_size=2, stride=2) # 16x16x128
        x = F.max_pool2d(F.relu(self.bn5(self.conv5(x))), kernel_size=2, stride=2) # 8x8x256
        x = F.max_pool2d(F.relu(self.bn6(self.conv6(x))), kernel_size=2, stride=2) # 4x4x512
        x = F.max_pool2d(F.relu(self.bn7(self.conv7(x))), kernel_size=2, stride=2) # 2x2x1024

        # Global average pooling across each channel (Input could be 2x2x256, 4x4x256, 7x3x256, output would always be 256 length vector)
        x = F.adaptive_avg_pool2d(x, 1)                                            # 1x1x1024
        x = torch.flatten(x, 1)                                                    # vector 1024
        
        x = self.fc1(x)
        return x

In [ ]:
def train(net, dataloader, epochs=1, start_epoch=0, lr=0.01, momentum=0.9, decay=0.0005, 
          verbose=1, print_every=10, state=None, schedule={}, checkpoint_path=None):
    net.to(device)
    net.train()
    losses = []
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=decay)


    for epoch in range(start_epoch, epochs):
        sum_loss = 0.0

        # Update learning rate when scheduled
        if epoch in schedule:
            print ("Learning rate: %f"% schedule[epoch])
            for g in optimizer.param_groups:
                g['lr'] = schedule[epoch]

        for i, batch in enumerate(dataloader, 0):
            inputs, labels = batch[0].to(device), batch[1].to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # autograd magic, computes all the partial derivatives
            optimizer.step() # takes a step in gradient direction

            losses.append(loss.item())
            sum_loss += loss.item()

            if i % print_every == print_every-1:    # print every 10 mini-batches
                if verbose:
                    print('[%d, %5d] loss: %.3f' % (epoch, i + 1, sum_loss / print_every))
                sum_loss = 0.0
            
    return losses

def evaluate(net, dataloader):
    net.to(device)
    net.eval()
    count = 1
    df = pd.DataFrame(columns=['path', "class"])
    with torch.no_grad():
        for batch in dataloader:
            print("batch:", count)
            count += 1
            images = batch[0].to(device)
            names = batch[1]
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            predicted = predicted.cpu()
            for i in range(len(predicted)):
                df = df.append({"path": "test/" + names[i], "class" : predicted[i].item()}, ignore_index=True)
            
    return df
def smooth(x, size):
    return np.convolve(x, np.ones(size)/size, mode='valid')

In [ ]:
final_net = ConvBNNet()
#Test Training
#final_losses =  train(final_net, data['train'], epochs=1, lr=.1  , decay=.0005)

#Real Training
final_losses =  train(final_net, data['train'], epochs=35, lr=.1  , decay=.0005)
final_losses += train(final_net, data['train'], epochs=20, lr=.01 , decay=.0005)
final_losses += train(final_net, data['train'], epochs=15, lr=.001, decay=.0005)
final_losses += train(final_net, data['train'], epochs=3, lr=.0001, decay=.0005)

In [ ]:
plt.plot(smooth(final_losses,50))


In [ ]:
# print("Testing  accuracy: %f" % accuracy(net, data['test']))
result = evaluate(final_net, data["test"])
print(result)  
# total = 0
# correct = 0
# with torch.no_grad():
#     for i, batch in enumerate(data["train"], 0):
#         images, labels = batch[0].to(device), batch[1].to(device)
#         outputs = net(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         print(i)


In [ ]:
result.to_csv("output.csv", index = False)

In [ ]:
len(result[result["class"] != 246])